In [2]:
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import numpy as np
import shap as shap

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import cross_validate, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from scipy.stats import loguniform
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix

## read data 
CIP_data_no_drop = pd.read_csv("CIP_data_encode_prev_not_dropped.csv")

/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
### set up model 
model_nn = MLPClassifier(solver = 'lbfgs', activation = 'tanh', max_iter = 3000 ,hidden_layer_sizes= 13, alpha = 1.291549665014884, random_state=10, learning_rate = 'constant' )
#train data: 2000 - 2010 
train_data = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,2010])]
X_train = train_data[['MSM','MSMW', 'MSW','Northeast', 'Southeast', 'West', 'Midwest','PREV_REGION', 'PREV_CLINIC']]
y_train = train_data['Susceptible']

oversample = RandomOverSampler(sampling_strategy = 0.5,random_state=42)
X_train, y_train = oversample.fit_resample(X_train,y_train)


#test data: 2011 
test_data = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([2011])]
X_test = test_data[['MSM','MSMW', 'MSW','Northeast', 'Southeast', 'West', 'Midwest','PREV_REGION', 'PREV_CLINIC']]
X_test.columns = ['MSM','MSMW', 'MSW','Northeast', 'Southeast', 'West', 'Midwest','PREV_REGION', 'PREV_CLINIC']
y_test = test_data['Susceptible']
cipro_R_prev_2011 = 1 - y_test.sum()/len(y_test) #get prevalence in 2011 before radndom oversampling
oversample = RandomOverSampler(sampling_strategy = 0.5,random_state=42)
X_test, y_test = oversample.fit_resample(X_test,y_test)


#fit model
model_fit_train = model_nn.fit(X_train, y_train)
y_predict_test = model_fit_train.predict(X_test)
y_predict_proba = model_fit_train.predict_proba(X_test)

In [14]:
### Get shapley values

explainer = shap.KernelExplainer(model_fit_train.predict, X_test)
shap_values = explainer.shap_values(X_test)

shap.plots.waterfall(shap_values[0])

X does not have valid feature names, but MLPClassifier was fitted with feature names
Using 7107 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
  0%|          | 0/7107 [00:00<?, ?it/s]X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted with feature names
  0%|          | 1/7107 [00:05<10:06:44,  5.12s/it]X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted with feature names
  0%|          | 2/7107 [00:10<10:00:50,  5.07s/it]X does not have valid feature names, but MLPClassifier was fitted with feature names
X does not have valid feature names, but MLPClassifier was fitted with feature names
  0%|          | 3/7107 [00:15<9:56:26,  5.04s/it] X does not have valid feature names, but MLPC

AttributeError: 'numpy.ndarray' object has no attribute 'base_values'

In [16]:
shap.plots.waterfall(shap_values)

AttributeError: 'numpy.ndarray' object has no attribute 'base_values'